In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
print("train_df.shape = ", train_df.shape)
train_df.head(2)

train_df.shape =  (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../07_Embeddings/exp/result/07_v1_01/oof_df.csv',1), # svr
    
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds     
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    #('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    #('../../14_Baseline4/exp/result/14_v1_17/oof_df.csv',1), # deberta-v3-large, 10folds
    
    #('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv2-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    #('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    #('../../19_Distillation/exp/result/19_v1_05/oof_df.csv',1), # deberta-v3-base <- db-l
    
    # single models
    #('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_04/oof_df.csv',1), # deberta-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_05/oof_df.csv',1), # deberta-v3-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_06/oof_df.csv',1), # deberta-v3-base <- db-l
    #('../../20_Distillation2/exp/result/20_v2_07/oof_df.csv',1), # deberta-v3-base <- svr
    #('../../20_Distillation2/exp/result/20_v2_08/oof_df.csv',1), # deberta-v3-base <- xlm-rb-b
    #('../../20_Distillation2/exp/result/20_v2_09/oof_df.csv',1), # deberta-v3-base <- xlm-rb-l
    #('../../20_Distillation2/exp/result/20_v2_10/oof_df.csv',1), # deberta-v3-base <- dbv2-xl
    #('../../20_Distillation2/exp/result/20_v2_11/oof_df.csv',1), # deberta-v3-base <- dbv3-b, 10folds
]

tmp_df = pd.read_csv('../../14_Baseline4/exp/result/14_v1_17/oof_df.csv').dropna()
text_ids = tmp_df["text_id"].values

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
        
oof_df_list = [oof_df.loc[oof_df["text_id"].isin(text_ids)].reset_index(drop=True) for oof_df in oof_df_list]
          
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
train_df = train_df.loc[train_df["text_id"].isin(text_ids)].reset_index(drop=True)

In [6]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [7]:
oof_df.shape, train_df.shape

((1955, 19), (1955, 8))

In [8]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df.loc[train_df["text_id"].isin(oof_df["text_id"]), TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4411


# Optimize class-wise

In [9]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(7, 1955, 6)

In [10]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [11]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0031
Function value obtained: 0.4726
Current minimum: 0.4726
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: 0.4712
Current minimum: 0.4712
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4730
Current minimum: 0.4712
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4714
Current minimum: 0.4712
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.3525
Function value obtained: 0.4703
Current minimum: 0.4701
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.3890
Function value obtained: 0.4710
Current minimum: 0.4701
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.3552
Function value obtained: 0.4714
Current minimum: 0.4701
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.3552
Function value obtained: 0.4712
Current minimum: 0.4701
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.3775
Function value obtained: 0.4724
Current minimum: 0.4701
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5724
Function value obtained: 0.4726
Current minimum: 0.4701
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.5847
Function value obtained: 0.4701
Current minimum: 0.4701
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.5752
Function value obtained: 0.4713
Current minimum: 0.4701
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.5747
Function value obtained: 0.4717
Current minimum: 0.4701
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.5740
Function value obtained: 0.4701
Current minimum: 0.4701
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2455
Function value obtained: 0.4338
Current minimum: 0.4330
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2424
Function value obtained: 0.4331
Current minimum: 0.4330
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.2771
Function value obtained: 0.4343
Current minimum: 0.4330
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.2867
Function value obtained: 0.4338
Current minimum: 0.4330
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.2865
Function value obtained: 0.4337
Current minimum: 0.4330
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.3905
Function value obtained: 0.4343
Current minimum: 0.4328
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.4231
Function value obtained: 0.4337
Current minimum: 0.4328
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.4284
Function value obtained: 0.4347
Current minimum: 0.4328
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.4681
Function value obtained: 0.4343
Current minimum: 0.4328
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.4627
Function value obtained: 0.4338
Current minimum: 0.4328
Iteration No: 66 started. Sea

...

Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.2518
Function value obtained: 0.4610
Current minimum: 0.4575
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.2198
Function value obtained: 0.4603
Current minimum: 0.4575
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.2033
Function value obtained: 0.4592
Current minimum: 0.4575
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.1941
Function value obtained: 0.4590
Current minimum: 0.4575
Iteration No: 15 started. Searching for the next optimal point.
Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 0.3416
Function value obtained: 0.4571
Current minimum: 0.4571
Iteration No: 16 started. Sea

Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.4117
Function value obtained: 0.4591
Current minimum: 0.4564
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.3861
Function value obtained: 0.4581
Current minimum: 0.4564
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.4025
Function value obtained: 0.4588
Current minimum: 0.4564
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.4068
Function value obtained: 0.4564
Current minimum: 0.4564
Iteration No: 55 started. Searching for the next optimal point.
Iteration No: 55 ended. Search finished for the next optimal point.
Time taken: 0.4388
Function value obtained: 0.4564
Current minimum: 0.4564
Iteration No: 56 started. Sea

Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.6883
Function value obtained: 0.4591
Current minimum: 0.4564
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.6892
Function value obtained: 0.4564
Current minimum: 0.4564
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.7357
Function value obtained: 0.4597
Current minimum: 0.4564
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.7053
Function value obtained: 0.4564
Current minimum: 0.4564
Iteration No: 95 started. Searching for the next optimal point.
Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 0.7212
Function value obtained: 0.4564
Current minimum: 0.4564
Iteration No: 96 started. Sea

Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3173
Function value obtained: 0.4325
Current minimum: 0.4321
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.3212
Function value obtained: 0.4325
Current minimum: 0.4321
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.2914
Function value obtained: 0.4328
Current minimum: 0.4321
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.3510
Function value obtained: 0.4322
Current minimum: 0.4321
Iteration No: 35 started. Searching for the next optimal point.
Iteration No: 35 ended. Search finished for the next optimal point.
Time taken: 0.3829
Function value obtained: 0.4329
Current minimum: 0.4321
Iteration No: 36 started. Sea

Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5367
Function value obtained: 0.4330
Current minimum: 0.4321
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.5479
Function value obtained: 0.4321
Current minimum: 0.4321
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.5461
Function value obtained: 0.4320
Current minimum: 0.4320
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.5678
Function value obtained: 0.4330
Current minimum: 0.4320
Iteration No: 75 started. Searching for the next optimal point.
Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 0.5718
Function value obtained: 0.4324
Current minimum: 0.4320
Iteration No: 76 started. Sea

In [12]:
best_weights

{'cohesion': array([0.03475642, 0.21969661, 0.15670204, 0.03475642, 0.34756416,
        0.03475642, 0.17176794]),
 'syntax': array([0.03213716, 0.19089737, 0.22194079, 0.04957821, 0.32137165,
        0.15193765, 0.03213716]),
 'vocabulary': array([0.17291648, 0.27792324, 0.27792324, 0.10152989, 0.1141225 ,
        0.02779232, 0.02779232]),
 'phraseology': array([0.04270131, 0.42701312, 0.17607716, 0.07776646, 0.11076887,
        0.12297177, 0.04270131]),
 'grammar': array([0.04045839, 0.22794253, 0.40458386, 0.04045839, 0.06085934,
        0.04045839, 0.18523912]),
 'conventions': array([0.02947607, 0.27200996, 0.24464213, 0.1186962 , 0.17269366,
        0.09127457, 0.07120742])}

In [13]:
scores

{'cohesion': 0.4700978393625958,
 'syntax': 0.4328409041366797,
 'vocabulary': 0.40365877650464443,
 'phraseology': 0.4435944680478021,
 'grammar': 0.456404577478157,
 'conventions': 0.432027550993679}

In [14]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4398


# Error Analysis - Check Corr

In [15]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.698768,0.677239,0.685395,0.641161,0.668906
syntax,0.698768,1.000000,0.673789,0.720754,0.712671,0.701790
vocabulary,0.677239,0.673789,1.000000,0.731118,0.650333,0.658787
phraseology,0.685395,0.720754,0.731118,1.000000,0.722669,0.668932
grammar,0.641161,0.712671,0.650333,0.722669,1.000000,0.676926
conventions,0.668906,0.701790,0.658787,0.668932,0.676926,1.000000


In [16]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.943184,0.917214,0.904503,0.822721,0.884248
syntax,0.943184,1.000000,0.937323,0.958387,0.916227,0.924695
vocabulary,0.917214,0.937323,1.000000,0.957578,0.878498,0.891225
phraseology,0.904503,0.958387,0.957578,1.000000,0.945265,0.878213
grammar,0.822721,0.916227,0.878498,0.945265,1.000000,0.850081
conventions,0.884248,0.924695,0.891225,0.878213,0.850081,1.000000
